In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, axis=-1) / 255.0  # Normalize pixel values
x_test = np.expand_dims(x_test, axis=-1) / 255.0
y_train = to_categorical(y_train, num_classes=10)  # One-hot encode labels
y_test = to_categorical(y_test, num_classes=10)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=0)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)





Epoch 1/5


750/750 [==============================] - 14s 16ms/step - loss: 0.2089 - accuracy: 0.9358 - val_loss: 0.0760 - val_accuracy: 0.9765
Epoch 2/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0580 - accuracy: 0.9820 - val_loss: 0.0523 - val_accuracy: 0.9844
Epoch 3/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0394 - accuracy: 0.9876 - val_loss: 0.0422 - val_accuracy: 0.9878
Epoch 4/5
750/750 [==============================] - 13s 17ms/step - loss: 0.0309 - accuracy: 0.9908 - val_loss: 0.0396 - val_accuracy: 0.9884
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0320 - accuracy: 0.9898
Test accuracy: 0.989799976348877


In [80]:
n = []
weight1 = model.get_weights()
weights = model.get_weights()
weight2 = model.get_weights()
weight3 = model.get_weights()
weight4 = model.get_weights()
weight5 = model.get_weights()
weight6 = model.get_weights()
weight7 = model.get_weights()
kk = [weights, weight1, weight2, weight3, weight4, weight5,
       weight6, weight7]
w = 0
for i in range(8):
    for j in range(8):
        w += kk[j][i]
    n.append(w)
    w = 0
n[7]

array([-0.05433574,  0.20077206, -0.02419269,  0.08503552, -0.16507953,
        0.30776837, -0.24675739, -0.01265088, -0.04276694, -0.07303198],
      dtype=float32)

In [14]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Define the local CNN model
def create_local_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Train multiple local models
num_local_models = 2
local_models = [create_local_model() for _ in range(num_local_models)]
for model in local_models:
    model.fit(x_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

# Combine model weights by averaging
global_weights = []
for layer_idx in range(len(local_models[0].get_weights())):
    layer_weights = [model.get_weights()[layer_idx] for model in local_models]
    layer_average_weights = np.mean(layer_weights, axis=0)
    global_weights.append(layer_average_weights)

# Create a global model with averaged weights
global_model = create_local_model()
global_model.set_weights(global_weights)

# Evaluate the global model
test_loss, test_accuracy = global_model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Epoch 1/2
750/750 [==============================] - 13s 16ms/step - loss: 0.2037 - accuracy: 0.9385 - val_loss: 0.0735 - val_accuracy: 0.9796
Epoch 2/2
750/750 [==============================] - 13s 17ms/step - loss: 0.0613 - accuracy: 0.9813 - val_loss: 0.0647 - val_accuracy: 0.9824
Epoch 1/2
750/750 [==============================] - 15s 18ms/step - loss: 0.2037 - accuracy: 0.9383 - val_loss: 0.0679 - val_accuracy: 0.9804
Epoch 2/2
750/750 [==============================] - 13s 17ms/step - loss: 0.0567 - accuracy: 0.9822 - val_loss: 0.0543 - val_accuracy: 0.9843
Test Loss: 1.5404547452926636
Test Accuracy: 0.775600016117096


In [32]:
len(local_models[0].get_weights())

8

In [29]:
for layer_idx in range(len(local_models[0].get_weights())):
    layer_weights = [model.get_weights()[layer_idx] for model in local_models]
    layer_average_weights = np.mean(layer_weights, axis=0)
    global_weights.append(layer_average_weights)
print(layer_average_weights)
global_weights

[-5.5341739e-03  2.7820321e-02 -1.0994218e-02  7.8351023e-03
 -2.8149430e-02  1.9055814e-02 -1.5192226e-02  4.2721694e-03
  4.4883694e-05 -8.1590209e-03]


[array([[[[-0.072983  , -0.03275631, -0.1072081 , -0.10788899,
           -0.06307754,  0.00141327,  0.03848143, -0.00262933,
           -0.0739884 ,  0.08764531,  0.05115508,  0.03459658,
           -0.030625  , -0.22423255,  0.00971382, -0.00874731,
            0.01178457,  0.13503109, -0.04341918, -0.03691459,
            0.15749696,  0.10470209, -0.05159912,  0.02760674,
            0.04670298, -0.11109212, -0.05298586, -0.09627672,
            0.1317909 ,  0.00741143,  0.14780706,  0.03673151]],
 
         [[-0.20952381,  0.0594715 , -0.08396156,  0.07781763,
            0.12281722, -0.01923816,  0.19109666,  0.12367772,
           -0.00743249,  0.00584441, -0.09940669, -0.08095032,
           -0.11386141, -0.09357385,  0.10538556,  0.03534646,
           -0.06569508,  0.02093276, -0.02922726, -0.01751959,
            0.03040775, -0.00267728, -0.02170255,  0.0130156 ,
            0.11596676, -0.08477625, -0.00534803,  0.05424216,
            0.0538973 , -0.02134688, -0.01529083,  

In [31]:
round(0.9751833279927572, 2)

0.98

In [15]:
len(local_models[0].get_weights())

8

In [10]:
weights = model.get_weights

<bound method Functional.get_weight_paths of <keras.src.engine.sequential.Sequential object at 0x000002585E96E250>>

In [3]:
import time

stTime = time.time()

for i in range(10000000):
    i*i

endTime = time.time()

print(f"{endTime-stTime}")

1.0757136344909668


In [4]:
l = [1,2,3,4]
sum(l)

10